# 2. Model Analysis and Comparison

This notebook loads the pre-trained models and evaluates their performance on the test set. It performs the following steps:
1. Loads and preprocesses the same historical data to recreate the exact same test set that the models were not trained on.
2. Loads the trained model objects (`baseline_model.pkl` and `2hl_model.pkl`) from the `../models/` directory.
3. Generates predictions from both models on the training, validation, and test sets.
4. Calculates the final Mean Squared Error (MSE) for each model on each dataset.
5. Visualizes the predictions against the actual values and presents a summary table.

### 2.1 Setup and Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src directory to path to import neural_networks module
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from neural_networks import NeuralNetwork_0hl, NeuralNetwork_2hl

# Set random seed for reproducibility
np.random.seed(42)

# Data Loading (same as in training notebook to ensure consistency)
df_pred_raw = pd.read_excel("../data/AR6-SYR-LR-F2-5-Panel(a).xlsx", sheet_name="Data")
df_pred = df_pred_raw.drop([1,2,3,4,6,7,8,10,11,13,14,16])
df_pred = df_pred.drop(['Unnamed: 1',2019], axis=1)
GHG_past_raw = df_pred.iloc[[0]].values[0,1:7]
df_pred = df_pred.rename(columns={'spm_cat (year)': 'Year'})
df_pred = df_pred.set_index('Year')
df_pred = df_pred.transpose()
df_pred = df_pred.drop('Past GHG emissions (Black line) ', axis=1)
df_pred = df_pred.rename(columns={'Trend from implemented policies (Lowest bound of  red shading ) ': 'Trend from implemented policies','Limit warming to 2°C (>67%) or return warming to 1.5°C (>50%) after a high overshoot, NDCs until 2030 (Median , dark navy blue line )': 'Limit warming to 2°C or return warming to 1.5°C after a high overshoot', 'Limit warming to 2°C (>67%) (Median , dark green line )': 'Limit warming to 2°C', 'Limit warming to 1.5°C (>50%) with no or limited overshoot ( Median ligh blue line ) ': 'Limit warming to 1.5°C'})
df_pred = df_pred.drop([2010,2011,2012,2013,2014], axis=0)
for year in df_pred.index[:-1]:
    diff = df_pred.loc[year+5] - df_pred.loc[year]
    for i in range(4):
        df_pred.loc[year+i+1] = df_pred.loc[year] + (i+1)*diff/5
df_pred = df_pred.sort_index()
df_past_GHG2 = pd.read_csv("https://ourworldindata.org/grapher/total-ghg-emissions.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
df_past_GHG2 = df_past_GHG2.loc[df_past_GHG2['Entity'] == 'World']
df_past_GHG2 = df_past_GHG2.drop(['Entity','Code'], axis=1)
df_past_GHG2 = df_past_GHG2.set_index('Year')
df_past_GHG2.annual_emissions_ghg_total_co2eq *= 10**(-9)
GHG_past_comb = df_past_GHG2.copy()
GHG_past_comb.loc[[2012, 2013, 2014], 'annual_emissions_ghg_total_co2eq'] = [float(emi) for emi in GHG_past_raw[2:5]]
del_years = np.arange(2015,2024)
GHG_past_comb = GHG_past_comb.drop(del_years)
df_sealevel = pd.read_csv("https://ourworldindata.org/grapher/sea-level.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
df_sealevel = df_sealevel.drop(['Entity','Code','sea_level_church_and_white_2011','sea_level_average'], axis=1)
df_sealevel = df_sealevel.dropna()
df_sealevel['Day'] = [np.datetime64(day) for day in df_sealevel['Day']]
df_sealevel = df_sealevel.groupby(df_sealevel.Day.dt.year).mean()
df_sealevel = df_sealevel.drop('Day', axis=1, errors='ignore')

# Normalization 
GHG_past_norm = (GHG_past_comb - GHG_past_comb.mean()) / GHG_past_comb.std()
sealevel_norm = (df_sealevel - df_sealevel.mean()) / df_sealevel.std()

# Sequence and Splitting 
def get_GHG_sequence(n_years, df_GHG, start_year, end_year):
    X, y = list(), list()
    for i in range(start_year, end_year + 1):
        end_ix = i - 1
        start_ix = end_ix - n_years + 1
        seq_x = df_GHG.loc[start_ix:end_ix]
        X.append(seq_x.to_numpy())
        y.append(sealevel_norm.loc[i].values)
    return np.array(X), np.array(y)

timespan = 15
train_end_year = 2000
validation_end_year = 2007
test_end_year = 2014

X_train, y_train = get_GHG_sequence(timespan, GHG_past_norm, 1970, train_end_year)
X_val, y_val = get_GHG_sequence(timespan, GHG_past_norm, train_end_year + 1, validation_end_year)
X_test, y_test = get_GHG_sequence(timespan, GHG_past_norm, validation_end_year + 1, test_end_year)

### 2.2 Load Pre-trained Models

In [ ]:
# Load the models from the files
nn_base = NeuralNetwork_0hl.load_model('../models/baseline_model.pkl')
nn_2hl = NeuralNetwork_2hl.load_model('../models/2hl_model.pkl')

print('Models loaded successfully.')

### 2.3 Evaluate Models

In [ ]:
# Generate predictions for all datasets
pred_base_train = nn_base.predict(np.squeeze(X_train))
pred_base_val = nn_base.predict(np.squeeze(X_val))
pred_base_test = nn_base.predict(np.squeeze(X_test))

pred_2hl_train = nn_2hl.predict(np.squeeze(X_train))
pred_2hl_val = nn_2hl.predict(np.squeeze(X_val))
pred_2hl_test = nn_2hl.predict(np.squeeze(X_test))

# Calculate MSE
mse_base_train_final = np.mean(np.square(y_train - pred_base_train))
mse_base_val_final = np.mean(np.square(y_val - pred_base_val))
mse_base_test_final = np.mean(np.square(y_test - pred_base_test))

mse_2hl_train_final = np.mean(np.square(y_train - pred_2hl_train))
mse_2hl_val_final = np.mean(np.square(y_val - pred_2hl_val))
mse_2hl_test_final = np.mean(np.square(y_test - pred_2hl_test))

print('--- Baseline Model ---')
print(f'Final MSE (Train): {mse_base_train_final:.4f}')
print(f'Final MSE (Validation): {mse_base_val_final:.4f}')
print(f'Final MSE (Test): {mse_base_test_final:.4f}')

print('
--- 2-Layer Model ---')
print(f'Final MSE (Train): {mse_2hl_train_final:.4f}')
print(f'Final MSE (Validation): {mse_2hl_val_final:.4f}')
print(f'Final MSE (Test): {mse_2hl_test_final:.4f}')

### 2.4 Comparison and Visualization

In [ ]:
# Plotting predictions vs actuals for the test set
plt.figure(figsize=(14, 6))

test_years = np.arange(validation_end_year + 1, test_end_year + 1)

plt.plot(test_years, y_test, label='Actual Sea Level', marker='o')
plt.plot(test_years, pred_base_test, label='Baseline Predictions', marker='x')
plt.plot(test_years, pred_2hl_test, label='2-Layer Predictions', marker='s')

plt.title('Model Predictions vs. Actual Data on Test Set')
plt.xlabel('Year')
plt.ylabel('Normalized Sea Level')
plt.legend()
plt.grid(True)
plt.show()

# Results Table
results = {
    'Model': ['Baseline (Linear)', '2-Hidden-Layer (ReLU)'],
    'Train MSE': [mse_base_train_final, mse_2hl_train_final],
    'Validation MSE': [mse_base_val_final, mse_2hl_val_final],
    'Test MSE': [mse_base_test_final, mse_2hl_test_final]
}
df_results = pd.DataFrame(results)
print(df_results.to_markdown(index=False))

### 2.5 Critique and Next Steps

**Analysis of Results:**

*(This section will be filled in after running the notebook, but we expect to see the 2-layer model outperform the baseline, indicating a non-linear relationship. We will also analyze the gap between training and test MSE to diagnose overfitting.)*

**Discussion of Open Points:**

*   **Overfitting:** If the 2-layer model shows a significantly lower training MSE than test MSE, it's a sign of overfitting. Future iterations should include regularization techniques like L2 regularization (weight decay) or dropout to combat this.
*   **Batch Size:** We are currently using batch gradient descent. Switching to mini-batch gradient descent would be computationally more efficient and could lead to better convergence by escaping local minima.
*   **Hyperparameters:** The architecture (15-8-4-1) was chosen arbitrarily. A systematic hyperparameter search (e.g., using grid search or random search) for the number of layers, number of neurons, learning rate, and epochs is a crucial next step.
*   **Data:** The primary limitation is likely the single-feature input. To build a more powerful model, we must enrich our dataset with more features. Potential candidates include global average temperature, atmospheric CO2 concentrations, and data on volcanic and solar activity. Sourcing this data, for example from NASA's public repositories, would be the most impactful next step.